In [2]:
#Load some pacakges to extract coast coordinates, manage data and calculate distances
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
from scipy import spatial
from sklearn.neighbors import NearestNeighbors
from pyproj import Proj, transform
import os
import pandas as pd

In [3]:
m = Basemap(epsg = '4326', resolution = 'i')
coast = m.drawcoastlines()
coordinates = np.vstack(coast.get_segments())
lons,lats = m(coordinates[:,0],coordinates[:,1])

In [4]:
print(coordinates[0:10])

[[-180.           68.99377441]
 [-179.62835693   68.89622498]
 [-179.24169922   68.85372162]
 [-178.4609375    68.58258057]
 [-178.71092224   68.67580414]
 [-178.71755981   68.54413605]
 [-176.75921631   67.98999786]
 [-176.96847534   67.97872162]
 [-176.78503418   67.97541809]
 [-176.78503418   67.97541809]]


In [5]:
from sqlalchemy import *
from geoalchemy2 import *
from sqlalchemy import func
from sqlalchemy.engine.url import URL
from sqlalchemy.orm import *

#Create connection with the Database (db = wef)

import database #Import Python file with a dict with the file configurations

def db_connect():
    """
    Performs database connection using database settings from settings.py.
    Returns sqlalchemy engine instance
    """
    return create_engine(URL(**database.DATABASE))


# Create the engine and the session connection to the Database
engine = db_connect()
connection = engine.connect()

#Load table (ais_position) from SQL database
metadata = MetaData()
ais_position = Table('ais_position', 
                 metadata, autoload=True,
                 schema = "ais_messages",
                 autoload_with=engine)

#Print variable names and len
print(ais_position.columns)
#print(len(ais_position.columns))

# Open a bound session on the database. This facilitates work and create a better workflow
Session = sessionmaker(bind=engine)
session = Session()

#Spatial query: only select AIS positional messages inside the Torres Strait
#wkt_string = "POLYGON((141.0205078125 -9.166331387642987, 143.602294921875 -9.155485188844034, 143.67919921875 -11.112885070321443, 140.965576171875 -11.11288507032144, 141.0205078125 -9.166331387642987))"
wkt_string =  "POLYGON((139.0 -9.0, 145.0 -9.0, 145 -12, 139 -12, 139 -9))"
q = select([ais_position], ais_position.c.geom.intersects(wkt_string))


['ais_position.msg_type', 'ais_position.mmsi', 'ais_position.timestamp', 'ais_position.nmea', 'ais_position.status', 'ais_position.rot', 'ais_position.speed', 'ais_position.accuracy', 'ais_position.longitude', 'ais_position.latitude', 'ais_position.course', 'ais_position.heading', 'ais_position.manever', 'ais_position.geom']


In [6]:
#Execture query with the connection object
results = connection.execute(q)
names_ais =  ais_position.columns
partial_results = results.fetchall()
results_df = pd.DataFrame(partial_results, columns=names_ais)

In [7]:
from scipy import stats

#Lon-lat
points = results_df.as_matrix([results_df.columns[8:10]])

def proj_arr(points,proj_to):
    """
    Project geographic co-ordinates to get cartesian x,y
    Transform(origin|destination|lon|lat)
    """
    inproj = Proj(init='epsg:4326')
    outproj = Proj(init=proj_to)
    func = lambda x: transform(inproj,outproj,x[0],x[1])
    return np.array(list(map(func, points)))

points_proj = proj_arr(points,'epsg:3410')
coordinates_proj = proj_arr(coordinates, 'epsg:3410')
print(points_proj)
print(coordinates_proj)

[[ 13588937.1838691   -1472348.45240297]
 [ 13547743.11396914  -1342219.45315338]
 [ 13831425.86495455  -1198671.21051533]
 ..., 
 [ 13706050.85019575  -1340626.6123294 ]
 [ 13629757.92659214  -1419733.86052764]
 [ 13841265.2670427   -1190674.29849698]]
[[-17334193.94368687   6867934.36090511]
 [-17298404.31595958   6863434.40498694]
 [-17261168.76141      6861467.5014896 ]
 ..., 
 [ -5211676.44974487  -6452824.24095462]
 [ -5218908.28801306  -6452704.26178031]
 [ -5220379.19527841  -6451961.63695584]]


In [8]:
#ScyPy failed approach with meter projection
distance,index = spatial.cKDTree(coordinates_proj).query(points_proj)
print(distance[1:100])
print(stats.describe(distance))

[ 139459.9071121    52307.99174647  201770.48946088  116151.3428111
   67276.60083692   12004.34339661   18492.65524901  205962.39288586
  113237.21150508  238837.51287728   43227.38401475   10436.39265207
   61636.32697261  227378.90851879  249636.4246947   175770.17434733
   94840.39793997  130602.6641289    10623.90169059   10296.89495917
   52618.69388298  176632.55203014   64155.4387509    51605.22070732
   10217.55608353  181419.61113791   16229.71045407   32902.5910045
   45875.94547784   44588.47056691   99056.94880344   95788.31636803
   80230.58015949   11632.83026479   51685.69012563  136819.40058509
  174184.54576047    7651.39684705   19173.67203852    8832.99385065
   93575.09990781   29365.43685498  216681.63464303    1868.3816467
  161018.38802352   59267.52216036   13813.55569421  245168.24732638
  182143.16892087   13433.58083644   67485.85845006    8835.69630128
  121843.81943643  207249.61519751    9948.49340041   18832.84742857
    4367.38197969   18346.56708441   

In [9]:
#Function to calculate distance from vessel to the nearest shore

def calculate_distance_to_shore(df, longitude, latitude):
    '''
    This function will create a numpy array of distances
    to shore. It will contain and ID for AIS points and 
    the distance to the nearest coastline point.
    '''
    
    def proj_arr(points,proj_to):
        """
        Project geographic co-ordinates to get cartesian x,y
        Transform(origin|destination|lon|lat) to meters.
        """
        inproj = Proj(init='epsg:4326')
        outproj = Proj(init=proj_to)
        func = lambda x: transform(inproj,outproj,x[0],x[1])
        return np.array(list(map(func, points)))
    
    if os.path.isfile('data/coast_coords.npy') == False: 
        '''
        Extract from Basemap coastlines all the coordinates
        to calculate distances between them and the AIS GPS
        points. The map is in intermediate resolution (i),
        this avoid having a detailed and -probably- lengthy
        calculation.
        '''
        m = Basemap(epsg = '4326', resolution = 'i')
        coast = m.drawcoastlines()
        coordinates = np.vstack(coast.get_segments())
        lons,lats = m(coordinates[:,0],coordinates[:,1],inverse=True)
        
        coordinates_proj = proj_arr(coordinates, 'epsg:3410')
        
        if not os.path.exists('data'):
            os.makedirs('data')
            np.save(os.path.join('data','coast_coords.npy'),coordinates_proj)
            
        else:
            np.save(os.path.join('data','coast_coords.npy'),coordinates_proj)
    
    #Load coast data
    print('Load coordinates')
    coast = np.load('data/coast_coords.npy')
    
    #Load coordinates from ais
    points = df.as_matrix([longitude, latitude])
    
    #Project to meters using 'proj_arr' function and calculate distance 
    points_proj = proj_arr(points, 'epsg:3410')
    distance,index = spatial.cKDTree(coast).query(points_proj)
    distance_km = distance/1000
        
    #Add new column to input dataframe
    df = df.assign(distance_to_shore = pd.Series(distance_km))
    return df



In [10]:
prueba = calculate_distance_to_shore(results_df, results_df.columns[8], results_df.columns[9])

print(coordinates[1:10])


for x, y  in zip(lons[1:10], lats[1:10]):
    print(x, y)

Load coordinates
[[-179.62835693   68.89622498]
 [-179.24169922   68.85372162]
 [-178.4609375    68.58258057]
 [-178.71092224   68.67580414]
 [-178.71755981   68.54413605]
 [-176.75921631   67.98999786]
 [-176.96847534   67.97872162]
 [-176.78503418   67.97541809]
 [-176.78503418   67.97541809]]
-179.628356934 68.8962249756
-179.241699219 68.8537216187
-178.4609375 68.5825805664
-178.710922241 68.6758041382
-178.717559814 68.5441360474
-176.759216309 67.9899978638
-176.968475342 67.9787216187
-176.78503418 67.9754180908
-176.78503418 67.9754180908


In [11]:
prueba

,ais_position.msg_type,ais_position.mmsi,ais_position.timestamp,ais_position.nmea,ais_position.status,ais_position.rot,ais_position.speed,ais_position.accuracy,ais_position.longitude,ais_position.latitude,ais_position.course,ais_position.heading,ais_position.manever,ais_position.geom,distance_to_shore
0,1,564856000,2017-06-11 14:48:35,"\c:1497192515*59\!AIVDM,1,1,,B,18Jd4h00Amb5tOQ...",0.0,0.0,11.7,1,141.108880,-11.544720,180.0,178.0,0.0,0101000020e610000008e6e8f17ba3614049f4328ae516...,92.667393
1,1,353518000,2017-04-29 15:58:09,"\c:1493481489*5E\!AIVDM,1,1,,A,15A91d002@:3w:u...",0.0,0.0,14.4,0,140.681117,-10.512183,287.0,284.0,0.0,0101000020e610000001032eb5cb9561402472d4e43c06...,139.459907
2,1,538005073,2017-05-21 23:22:00,"\c:1495408920*57\!AIVDM,1,1,,A,1815>D@02?bAN<Q...",0.0,0.0,14.3,1,143.626907,-9.377147,234.6,234.0,0.0,0101000020e610000049df919e0ff46140eb412e5e19c1...,52.307992
3,1,566626000,2017-05-22 23:58:42,"\c:1495497522*5F\!AIVDM,1,1,,B,18LH6l001rb0Sr1...",0.0,0.0,12.2,1,139.932693,-11.417680,129.9,129.0,0.0,0101000020e6100000a57bb09fd87d6140f6622827dad5...,201.770489
4,1,413245000,2017-06-12 11:47:56,"\c:1497268076*5F\!AIVDM,1,1,,B,16:6NB022;:E:pu...",0.0,2.0,13.9,0,144.434823,-9.408287,321.3,324.0,0.0,0101000020e61000008a869f12ea0d6240cf7331f30ad1...,116.151343
5,1,355701000,2017-05-04 07:00:36,"\c:1493881236*50\!AIVDM,1,1,,B,15C>A20027:7i`i...",0.0,0.0,13.5,0,141.508733,-11.517817,2.0,7.0,0.0,0101000020e6100000a8a1208b47b061406b5021441f09...,67.276601
6,1,636012596,2017-06-13 08:23:21,"\c:1497342201*54\!AIVDM,1,1,,A,1INS8=002=:>15i...",0.0,0.0,14.1,0,142.872200,-10.980300,340.0,343.0,0.0,0101000020e61000004772f90fe9db6140a089b0e1e9f5...,12.004343
7,1,373836000,2017-05-27 10:37:05,"\c:1495881425*52\!AIVDM,1,1,,B,15TQ8p001tb=Dhc...",0.0,0.0,12.4,1,142.720888,-10.766337,293.6,293.0,0.0,0101000020e610000082f7688411d76140341e92475d88...,18.492655
8,1,236494000,2017-05-05 03:18:01,"\c:1493954281*55\!AIVDM,1,1,,A,13QRLd0u1O:0h5?...",0.0,6.0,9.5,0,139.974252,-10.754560,103.0,99.0,0.0,0101000020e610000005cdd4112d7f614051bd35b05582...,205.962393
9,1,477669100,2017-05-30 12:57:51,"\c:1496149071*59\!AIVDM,1,1,,B,177RVs031nb5<>Q...",0.0,6.0,11.8,1,140.944133,-10.576800,103.1,106.0,0.0,0101000020e61000005db71b57369e614065aa60545227...,113.237212
